# Tools and Routing

In [ ]:
import os
import openai

from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv()) # read local .env file
openai.api_key = os.environ['OPENAI_API_KEY']

In [ ]:
from langchain.agents import tool

In [ ]:
# 함수를 langchain의 tool로 자동 변환해주는 decorator
@tool
def search(query: str) -> str:
    """Search for weather online"""
    return "42f"

In [ ]:
search.name

In [ ]:
search.description

In [ ]:
search.args

In [ ]:
# query라는 문자열이 포함되어야 하는 `SearchInput`을 정의합니다.
from pydantic import BaseModel, Field
class SearchInput(BaseModel):
    query: str = Field(description="Thing to search for")

`args_schema`에 `SearchInput`을 넣어주어 description을 상속합니다.

In [ ]:
@tool(args_schema=SearchInput)
def search(query: str) -> str:
    """Search for the weather online."""
    return "42f"

In [ ]:
search.args

In [ ]:
search.run("sf")

날씨 관련 정보를 확인할 수 있는 툴을 사용해 봅니다.

우선 `OpenMeteoInput`에서 input의 schema를 정해줍니다.

그리고 `get_current_temperature` 함수를 langchain의 tool로 만들며 input schema를 지정합니다.

In [ ]:
import requests
from pydantic import BaseModel, Field
import datetime

# Define the input schema
class OpenMeteoInput(BaseModel):
    latitude: float = Field(..., description="Latitude of the location to fetch weather data for")
    longitude: float = Field(..., description="Longitude of the location to fetch weather data for")

@tool(args_schema=OpenMeteoInput)
def get_current_temperature(latitude: float, longitude: float) -> dict:
    """Fetch current temperature for given coordinates."""
    
    BASE_URL = "https://api.open-meteo.com/v1/forecast"
    
    # request를 위한 파라미터
    params = {
        'latitude': latitude,
        'longitude': longitude,
        'hourly': 'temperature_2m',
        'forecast_days': 1,
    }

    # request를 보내고 결과를 받아옵니다.
    response = requests.get(BASE_URL, params=params)
    
    if response.status_code == 200:
        results = response.json()
    else:
        raise Exception(f"API Request failed with status code: {response.status_code}")

    current_utc_time = datetime.datetime.utcnow()
    time_list = [datetime.datetime.fromisoformat(time_str.replace('Z', '+00:00')) for time_str in results['hourly']['time']]
    temperature_list = results['hourly']['temperature_2m']
    
    closest_time_index = min(range(len(time_list)), key=lambda i: abs(time_list[i] - current_utc_time))
    current_temperature = temperature_list[closest_time_index]
    
    return f'The current temperature is {current_temperature}°C'

In [ ]:
get_current_temperature.name

In [ ]:
get_current_temperature.description

In [ ]:
get_current_temperature.args

langchain의 tool을 openai의 function으로 변환하는 함수를 불러옵니다.

In [ ]:
from langchain.tools.render import format_tool_to_openai_function

In [ ]:
format_tool_to_openai_function(get_current_temperature)

In [ ]:
get_current_temperature({"latitude": 13, "longitude": 14})

이번에는 날씨가 아닌 정보를 확인할 수 있도록 wikipedia를 활용해 봅니다.

In [ ]:
import wikipedia
@tool
def search_wikipedia(query: str) -> str:
    """Run Wikipedia search and get page summaries."""
    page_titles = wikipedia.search(query)
    summaries = []
    for page_title in page_titles[: 3]:
        try:
            wiki_page =  wikipedia.page(title=page_title, auto_suggest=False)
            summaries.append(f"Page: {page_title}\nSummary: {wiki_page.summary}")
        except (
            self.wiki_client.exceptions.PageError,
            self.wiki_client.exceptions.DisambiguationError,
        ):
            pass
    if not summaries:
        return "No good Wikipedia Search Result was found"
    return "\n\n".join(summaries)

In [ ]:
search_wikipedia.name

In [ ]:
search_wikipedia.description

In [ ]:
format_tool_to_openai_function(search_wikipedia)

In [ ]:
search_wikipedia({"query": "langchain"})

이번에는 spec에 관한 함수를 불러옵니다.

spec 정보를 직접 활용하여 openai의 function을 정의하는 방식입니다.

In [ ]:
from langchain.chains.openai_functions.openapi import openapi_spec_to_openai_fn
from langchain.utilities.openapi import OpenAPISpec

In [ ]:
text = """
{
  "openapi": "3.0.0",
  "info": {
    "version": "1.0.0",
    "title": "Swagger Petstore",
    "license": {
      "name": "MIT"
    }
  },
  "servers": [
    {
      "url": "http://petstore.swagger.io/v1"
    }
  ],
  "paths": {
    "/pets": {
      "get": {
        "summary": "List all pets",
        "operationId": "listPets",
        "tags": [
          "pets"
        ],
        "parameters": [
          {
            "name": "limit",
            "in": "query",
            "description": "How many items to return at one time (max 100)",
            "required": false,
            "schema": {
              "type": "integer",
              "maximum": 100,
              "format": "int32"
            }
          }
        ],
        "responses": {
          "200": {
            "description": "A paged array of pets",
            "headers": {
              "x-next": {
                "description": "A link to the next page of responses",
                "schema": {
                  "type": "string"
                }
              }
            },
            "content": {
              "application/json": {
                "schema": {
                  "$ref": "#/components/schemas/Pets"
                }
              }
            }
          },
          "default": {
            "description": "unexpected error",
            "content": {
              "application/json": {
                "schema": {
                  "$ref": "#/components/schemas/Error"
                }
              }
            }
          }
        }
      },
      "post": {
        "summary": "Create a pet",
        "operationId": "createPets",
        "tags": [
          "pets"
        ],
        "responses": {
          "201": {
            "description": "Null response"
          },
          "default": {
            "description": "unexpected error",
            "content": {
              "application/json": {
                "schema": {
                  "$ref": "#/components/schemas/Error"
                }
              }
            }
          }
        }
      }
    },
    "/pets/{petId}": {
      "get": {
        "summary": "Info for a specific pet",
        "operationId": "showPetById",
        "tags": [
          "pets"
        ],
        "parameters": [
          {
            "name": "petId",
            "in": "path",
            "required": true,
            "description": "The id of the pet to retrieve",
            "schema": {
              "type": "string"
            }
          }
        ],
        "responses": {
          "200": {
            "description": "Expected response to a valid request",
            "content": {
              "application/json": {
                "schema": {
                  "$ref": "#/components/schemas/Pet"
                }
              }
            }
          },
          "default": {
            "description": "unexpected error",
            "content": {
              "application/json": {
                "schema": {
                  "$ref": "#/components/schemas/Error"
                }
              }
            }
          }
        }
      }
    }
  },
  "components": {
    "schemas": {
      "Pet": {
        "type": "object",
        "required": [
          "id",
          "name"
        ],
        "properties": {
          "id": {
            "type": "integer",
            "format": "int64"
          },
          "name": {
            "type": "string"
          },
          "tag": {
            "type": "string"
          }
        }
      },
      "Pets": {
        "type": "array",
        "maxItems": 100,
        "items": {
          "$ref": "#/components/schemas/Pet"
        }
      },
      "Error": {
        "type": "object",
        "required": [
          "code",
          "message"
        ],
        "properties": {
          "code": {
            "type": "integer",
            "format": "int32"
          },
          "message": {
            "type": "string"
          }
        }
      }
    }
  }
}
"""

In [ ]:
spec = OpenAPISpec.from_text(text)

텍스트로부터 정의된 spec을 openai의 function 형태로 변환하며 return되는 값을 나누어 갖습니다.

In [ ]:
pet_openai_functions, pet_callables = openapi_spec_to_openai_fn(spec)

In [ ]:
pet_openai_functions

In [ ]:
from langchain.chat_models import ChatOpenAI

Chat 모델을 위에서 정의한 함수로 묶고 질문을 던져봅니다.

각 질문에 답변하기 적합한 함수가 호출되는 것을 확인할 수 있습니다.

In [ ]:
model = ChatOpenAI(temperature=0).bind(functions=pet_openai_functions)

In [ ]:
model.invoke("what are three pets names")

In [ ]:
model.invoke("tell me about pet with id 42")

### Routing

세 번째 강의에서는 두 개의 후보 function 중에서 적절한 것을 골라 호출하는 예시를 살펴보았습니다.

위에서 배운 tool을 활용하여 이를 OpenAI function 형식으로 만들어 보겠습니다.

In [ ]:
functions = [
    format_tool_to_openai_function(f) for f in [
        search_wikipedia, get_current_temperature
    ]
]
model = ChatOpenAI(temperature=0).bind(functions=functions)

질문에 어울리는 함수가 호출되는 것을 확인할 수 있습니다.

In [ ]:
model.invoke("what is the weather in sf right now")

In [ ]:
model.invoke("what is langchain")

In [ ]:
from langchain.prompts import ChatPromptTemplate
prompt = ChatPromptTemplate.from_messages([
    ("system", "You are helpful but sassy assistant"),
    ("user", "{input}"),
])
chain = prompt | model

In [ ]:
chain.invoke({"input": "what is the weather in sf right now"})

하지만 반환되는 형태가 알아보기에 좋지는 않기 때문에, 이를 OutputParser와 연결해줍니다.

In [ ]:
from langchain.agents.output_parsers import OpenAIFunctionsAgentOutputParser

In [ ]:
chain = prompt | model | OpenAIFunctionsAgentOutputParser()

In [ ]:
result = chain.invoke({"input": "what is the weather in sf right now"})

In [ ]:
type(result)

In [ ]:
result.tool

In [ ]:
result.tool_input

In [ ]:
get_current_temperature(result.tool_input)

In [ ]:
result = chain.invoke({"input": "hi!"})

In [ ]:
type(result)

In [ ]:
result.return_values

위에서 확인한 바에 따르면 parser를 통해 나온 것 중에 `Agentfinish`가 반환되어야 답변이 출력되는 것을 알 수 있습니다.

따라서 `route` 함수에는 `result`의 type이 `AgentFinish`가 맞는지에 따라 다른 값을 반환하도록 합니다.

In [ ]:
from langchain.schema.agent import AgentFinish
def route(result):
    if isinstance(result, AgentFinish):
        return result.return_values['output']
    else:
        tools = {
            "search_wikipedia": search_wikipedia, 
            "get_current_temperature": get_current_temperature,
        }
        return tools[result.tool].run(result.tool_input)

In [ ]:
chain = prompt | model | OpenAIFunctionsAgentOutputParser() | route

질문에 답변하기 적합한 함수들이 호출되어 반환된 결과가 깔끔하게 출력되는 것을 확인할 수 있습니다.

In [ ]:
result = chain.invoke({"input": "What is the weather in san francisco right now?"})

In [ ]:
result

In [ ]:
result = chain.invoke({"input": "What is langchain?"})

In [ ]:
result

In [ ]:
chain.invoke({"input": "hi!"})